In [25]:
import math
import numpy as np

from collections import OrderedDict
from matplotlib import pyplot as plt

In [26]:
## A Naive Congestion Pricing Genco model

class CPGenco:
    def __init__(self):
        self.seed_value = 0 # seed value will be overidden
        self.total_quantity = 1500 
        self.cleared_quantity = 0
        self.knee_demand = 900 # Congesiton threshold
        self.knee_slope = 5
        self.min_ask_quant = 15
        self.a = 0.00004132
        self.b = 0.00182
        self.c = 14
        self.p = 6
        
    def scale_coeff(self):
        y = self.a * self.knee_demand *self.knee_demand + self.b * self.knee_demand + self.c
        self.a *= self.knee_slope
        self.b *= self.knee_slope
        self.c = y - self.a * self.knee_demand * self.knee_demand - self.b * self.knee_demand        
        self.p *= 2
 
    def quad_function(self, cum_quantity):
        a,b,c,p = self.a,self.b,self.c,self.p
        if cum_quantity >= self.knee_demand: 
            self.scale_coeff()
                    
        q = cum_quantity
        price = self.a*(q**2)+self.b*q+self.c
        quantity = max((-self.b/(2.0*self.a) + math.sqrt(self.b**2+4.0*self.a*(self.a*(q**2)+self.b*q+self.p))/(2.0*self.a) - q) , self.min_ask_quant)
        self.a,self.b,self.c,self.p = a,b,c,p
        return price, quantity
    
    def asks(self):
        asks = dict()
        cum_quantity = self.cleared_quantity
        while cum_quantity < self.total_quantity:
            price, quantity = self.quad_function(cum_quantity)
            cum_quantity += quantity
            asks.update({price: -quantity})

        return asks
    
    def set_cleared_quantity(self, cleared_quantity):
        self.cleared_quantity += cleared_quantity
    
    def plot(self):
        ask_list = self.asks()
        prices = list(ask_list.keys())
        quantities = list(ask_list.values())
        cum_quantities = np.cumsum(quantities)
        plt.step(np.negative(cum_quantities), prices, where='post')

In [34]:
## A Randomized Way to Generate Buyer's Bids

class Buyer:
    def __init__(self, low, high, total_demand, number_of_bids):
        self.seed_value = 0 # seed value will be overidden
        self.total_demand = total_demand 
        self.cleared_demand = 0
        self.min_bid_quant = 0.01
        self.number_of_bids = number_of_bids
        self.low = low
        self.high = high
 
    def gen_function(self, rem_quantity):
        price = np.random.uniform(self.low, self.high)
        quantity = max(rem_quantity/self.number_of_bids, self.min_bid_quant)
        return price, quantity
    
    def bids(self):
        bids = dict()
        rem_quantity = self.total_demand - self.cleared_demand
        for i in range(self.number_of_bids):
            price, quantity = self.gen_function(rem_quantity)
            bids.update({-price: quantity})

        return dict(sorted(bids.items()))
    
    def set_cleared_demand(self, cleared_demand):
        self.cleared_demand += cleared_demand

In [39]:
def clearing_mechanism(self,actions):
        # actions are bids convert to list of bids
        
        actions_li = {k : v.tolist() for k, v in actions.items()}
        
        # Seperate bids from agent and have a mapping of player to bids
        bid_vals = []
        bid_player_map = []

        for key in actions_li.keys():
            for val in actions_li[key]:
                bid_vals.append(val)
                bid_player_map.append(key)
        
        bid_val_ind = sorted(range(len(bid_vals)), key=lambda k: (-bid_vals[k][0],-bid_vals[k][1]))
    
        bid_vals = [bid_vals[i] for i in bid_val_ind]    
        self.sorted_bids = np.array(bid_vals)

        if self.render_mode == "human":
            self.render()
        bid_player_map = [bid_player_map[i] for i in bid_val_ind]

        # Then sort the bids and the mapping
        uni_bids = {}
        for key, val in bid_vals:
            if key in uni_bids:
                uni_bids[key].extend([val])
            else:
                uni_bids[key] = [val] 
   
        # Unique price bids that is all bids with same are combined
        # Then pass the bids to combine the bids with the same price
        uni_bids_comb =[]
        for k,v in uni_bids.items():
            uni_bids_comb.append([k,sum(v)])

        # copy the ask and bid quantities
        cleared_asks_quant = [ask[1] for ask in self.asks]
        cleared_bids_quant = [bid[1] for bid in uni_bids_comb]

        i = self.last_cl_ask_index
        j = 0
        N_a = len(self.asks)
        N_b = len(uni_bids_comb)

        last_ask_index = self.last_cl_ask_index
        last_bid_index = N_b
        
        while j < N_b and i < N_a:
            if self.asks[i][0] > (uni_bids_comb[j][0]):
                break
            match_quant = min(self.asks[i][1],uni_bids_comb[j][1])
            self.asks[i][1] -= match_quant        
            uni_bids_comb[j][1] -= match_quant 
            last_ask_index, last_bid_index = i, j
            
            # Check if the quantity of particular ask is satisfied, 
            # if yes then go to next ask
            if self.asks[i][1] == 0:
                i += 1 

            # Check if the quantity of particular bid is satisfied, 
            # if yes then go to next bid
            if uni_bids_comb[j][1] == 0:
                j += 1
            
        cleared_asks_quant = cleared_asks_quant - self.asks[:,1]

        uni_bids_comb_column = [u[1] for u in uni_bids_comb]
        cleared_bids_quant = np.array(cleared_bids_quant) - np.array(uni_bids_comb_column)

        #Total cleared quantity
        mcq = sum(cleared_asks_quant) 

        # Clearing price
        if mcq == 0:
            self.mcp = 0 ## Need to verify this 
            
        # Average clearing price rule is used; However can be modified to get range of mcp
        else:
            self.mcp = (self.asks[last_ask_index][0]+uni_bids_comb[last_bid_index][0])/2 

        cleared_bids = []
        prices = list(uni_bids.keys()) ## ERROR ALMOST SAME NAME NEED TO CHANGE

        for i in range(len(prices)):
        # Assumes that quantities are in descending order for the same price
            if cleared_bids_quant[i] == 0: # Price not cleared
                for _ in uni_bids[prices[i]]:
                    cleared_bids.append([prices[i],0])                
            elif sum(uni_bids[prices[i]]) == cleared_bids_quant[i]: # Fully cleared
                for v in uni_bids[prices[i]]:
                    cleared_bids.append([prices[i],v])
            else: # Partially cleared; So need to distribute among equal price bids
                count = Counter(uni_bids[prices[i]])
                comb_quant = sorted(count.items(), key=lambda pair: pair[0], reverse=True)
                clear_quant_price = cleared_bids_quant[i]

                for j in range(len(comb_quant)):
                    
                    if clear_quant_price == 0: 
                        # Not cleared
                        for _ in range(comb_quant[j][1]):
                            cleared_bids.append([prices[i],0])
                    elif comb_quant[j][0] * comb_quant[j][1] <=  clear_quant_price:
                        # Fully cleared
                        for _ in range(comb_quant[j][1]):
                            cleared_bids.append([prices[i],comb_quant[j][0]])
                        clear_quant_price -= comb_quant[j][0] * comb_quant[j][1]
                    else:
                        # Partially cleared and Equally distributed
                        each_cleared_quant = clear_quant_price/comb_quant[j][1] # round() is removed
                        
                        for _ in range(comb_quant[j][1]):
                            cleared_bids.append([prices[i],each_cleared_quant])
                        #clear_quant_price -= comb_quant[j][0] * comb_quant[j][1] # might lead to numerical instability
                        # Hence set
                        clear_quant_price = 0

        # Calculate rewards or costs
        rewards = {}
        cleared_quant_agent = {}
        for key,  val in zip(bid_player_map,cleared_bids):
            if key in rewards:
                rewards[key] += (1 * self.mcp) * val[1] # <----- respresents cost hence negative
                cleared_quant_agent[key] += val[1]
            else:
                rewards[key] = (1* self.mcp) * val[1] # <------- Represents cost hence negative
                cleared_quant_agent[key] = val[1] 
        
        rewards = {k: v for k, v in sorted(list(rewards.items()))} # See better way to do this
        cleared_quant_agent = {k: v for k, v in sorted(list(cleared_quant_agent.items()))}

        if last_ask_index > -1 and last_ask_index < N_a:
            if self.asks[last_ask_index][1] == 0.0:
                self.last_cl_ask_index = last_ask_index + 1
                self.asks[last_ask_index][0] = 0
            else:
                self.last_cl_ask_index = last_ask_index

        for i in self.agents:
            self.requirements[i] = max(0.0,self.requirements[i]-cleared_quant_agent[i])
        
        return rewards

# Below Code is for One Auction Containting 24 Rounds

In [36]:
asks_list = dict()
bids_list = dict()

# GenCo's asks
genCo = CPGenco()
asks_list.update({"cp_genco": genCo.asks()})

market_demand = np.random.normal(60, 7)
bids_list.update({"miso": {"null": np.random.uniform(800, 100)}})

# Buyer's bids
buyer1 = Buyer(0, 100, market_demand*0.7, 10)
bids_list.update({"buyer1": buyer1.bids()})

buyer2 = Buyer(0, 100, market_demand*0.3, 10)
bids_list.update({"buyer2": buyer2.bids()})

In [40]:
# PDA simulator

rounds = 24
cur_round = 0

while(cur_round < rounds):
    
    proximity = rounds - cur_round    
    
    result = clearing_mechanism(asks_list, bids_list)
    
    genCo.set_cleared_quantity()
    buyer1.set_cleared_demand()
    buyer2.set_cleared_demand()
    
    # asks and bids for the next round     
    asks_list = dict()
    bids_list = dict()

    asks_list.update({"cp_genco": genCo.asks()})
    bids_list.update({"buyer1": buyer1.bids()})
    bids_list.update({"buyer2": buyer2.bids()})
    
    cur_round += 1

NameError: name 'clearing_mechanism' is not defined

# Testing Auction Clearing Mechanism

In [41]:
asks_list = dict()
bids_list = dict()

# GenCo's asks
genCo = CPGenco()
asks_list.update({"cp_genco": genCo.asks()})

market_demand = np.random.normal(60, 7)
bids_list.update({"miso": {"null": np.random.uniform(800, 100)}})

# Buyer's bids
buyer1 = Buyer(0, 100, market_demand*0.7, 10)
bids_list.update({"buyer1": buyer1.bids()})

buyer2 = Buyer(0, 100, market_demand*0.3, 10)
bids_list.update({"buyer2": buyer2.bids()})

In [42]:
asks_list, bids_list

({'cp_genco': {14.0: -359.67449325806183,
   20.0: -157.65492812589343,
   26.0: -121.03315510434186,
   32.0: -102.0560236815752,
   38.000000000000014: -89.92292476458317,
   44.000000000000014: -81.30210566041433,
   53.57120000000009: -30.603360366651714,
   65.57120000000003: -29.661522922081986,
   77.5712: -28.801634847871128,
   89.57119999999995: -28.012459446676303,
   101.57119999999989: -27.284805602774213,
   113.57119999999983: -26.611072531280342,
   125.5711999999998: -25.984912556833706,
   137.57119999999972: -25.40097737668384,
   149.57119999999972: -24.854724429214002,
   161.57119999999972: -24.342267232734912,
   173.57119999999966: -23.86025835992109,
   185.57119999999966: -23.40579695659403,
   197.57119999999966: -22.976354943569277,
   209.57119999999966: -22.569717598332318,
   221.57119999999966: -22.183935317972782,
   233.57119999999972: -21.817284158674283,
   245.5711999999996: -21.46823332476788,
   257.5711999999995: -21.13541820559908,
   269.571199

In [ ]:
# clearing_mechanism

bidPrice = 0.0;
askPrice = 0.0;
totalMWh = 0.0;

while (bids != null && !bids.isEmpty() &&
             asks != null && !asks.isEmpty() &&
             (bids.get(0).isMarketOrder() ||
                 asks.get(0).isMarketOrder() ||
                 -bids.get(0).getLimitPrice() >= asks.get(0).getLimitPrice())) 
{}